**Set Up**

In [36]:
# Import Libraries
# Build Model
import boto3
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import joblib
import time
import sagemaker
from sagemaker import get_execution_role

# Monitoring
from sagemaker.model_monitor import DefaultModelMonitor
from sagemaker.model_monitor.dataset_format import DatasetFormat
import json

# Monitoring Scheduling
from sagemaker import Session
from sagemaker.model_monitor import DefaultModelMonitor, DataCaptureConfig

role = get_execution_role()

In [2]:
# Load Data from S3
s3 = boto3.client('s3')
bucket = 'dmg-iris-hakkoda'
key = 'iris.csv'
obj = s3.get_object(Bucket=bucket, Key=key)
df = pd.read_csv(obj['Body'])

In [3]:
# Prep Data
X = df.drop('species', axis=1)
y = df['species']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

**Build and Evaluate Model**

In [4]:
# Train Model
model = RandomForestClassifier()
model.fit(X_train, y_train)

RandomForestClassifier()

In [5]:
# Evaluate Model
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Model accuracy: {accuracy}")

Model accuracy: 0.9666666666666667


**Save Model to S3**

In [6]:
# Save Model
joblib.dump(model, 'dmg-iris-model.joblib')

['dmg-iris-model.joblib']

In [7]:
# Upload Model to S3
s3.upload_file('dmg-iris-model.joblib', bucket, 'models/dmg-iris-model.joblib')

**Save Model to Model Registry**

In [14]:
# Create SageMaker Model Package Group (this is a container for versioned model packages)
# This step creates a Model Package Group in SageMaker, allowing multiple model versions to be stored and managed.

sagemaker_client = boto3.client("sagemaker")

model_package_group_name = "dmg-iris-package-group"  # Change this to your desired name

# Create the model package group
response = sagemaker_client.create_model_package_group(
    ModelPackageGroupName=model_package_group_name,
    ModelPackageGroupDescription="Model group for versioned SageMaker models"
)

print(f"Model Package Group ARN: {response['ModelPackageGroupArn']}")

Model Package Group ARN: arn:aws:sagemaker:us-east-1:878763418883:model-package-group/dmg-iris-package-group


In [15]:
# Create a Model Package
# Register it in the SageMaker Model Registry

# Explanation of Parameters:
# ModelPackageGroupName → The group to store your versioned models.
# ModelApprovalStatus → Can be "PendingManualApproval" or "Approved".
# InferenceSpecification → Defines how the model will be used for inference.

model_artifact_s3 = f"s3://{bucket}/models/dmg-iris-model.joblib"
inference_container_image = "683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-scikit-learn:1.2-1"  # Scikit-Learn container

response = sagemaker_client.create_model_package(
    ModelPackageGroupName=model_package_group_name,
    ModelPackageDescription="Registered locally trained model",
    ModelApprovalStatus="PendingManualApproval",
    InferenceSpecification={
        "Containers": [
            {
                "Image": inference_container_image,
                "ModelDataUrl": model_artifact_s3,
            }
        ],
        "SupportedContentTypes": ["application/json"],
        "SupportedResponseMIMETypes": ["application/json"],
    },
)

print(f"Model Package ARN: {response['ModelPackageArn']}")

Model Package ARN: arn:aws:sagemaker:us-east-1:878763418883:model-package/dmg-iris-package-group/1


In [17]:
# Check Model Status
response = sagemaker_client.list_model_packages(ModelPackageGroupName=model_package_group_name)

for model in response["ModelPackageSummaryList"]:
    print(f"Model Package Name: {model['ModelPackageArn']}")
    print(f"Status: {model['ModelApprovalStatus']}")
    print("----")


Model Package Name: arn:aws:sagemaker:us-east-1:878763418883:model-package/dmg-iris-package-group/1
Status: PendingManualApproval
----


In [18]:
# If needs manual approval:
model_package_arn = "arn:aws:sagemaker:us-east-1:878763418883:model-package/dmg-iris-package-group/1"

sagemaker_client.update_model_package(
    ModelPackageArn=model_package_arn,
    ModelApprovalStatus="Approved",
    ApprovalDescription="Model validated and approved for deployment."
)

print("Model successfully approved!")

Model successfully approved!


**Set up Model Monitoring**

In [25]:
# Set up Baseline
# Analyze a baseline dataset to understand the data schema, statistics, and constraints.

# Initialize SageMaker session and role
sagemaker_session = sagemaker.Session()
role = get_execution_role()

# Define variables
baseline_dataset_uri = 's3://dmg-iris-hakkoda/iris.csv'
baseline_output_uri = 's3://dmg-iris-hakkoda'

# Create a default model monitor instance
model_monitor = DefaultModelMonitor(
    role=role,
    sagemaker_session=sagemaker_session
)

# Create a baseline job
model_monitor.suggest_baseline(
    baseline_dataset=baseline_dataset_uri,
    dataset_format=DatasetFormat.csv(header=True),
    output_s3_uri=baseline_output_uri,
    wait=True
)

[02/04/25 21:00:21] INFO     Ignoring unnecessary instance type: None.                            ]8;id=855595;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=384632;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/image_uris.py#528\528]8;;\

                    INFO     Creating processing-job with name                                      ]8;id=805064;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=793131;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#1575\1575]8;;\
                             baseline-suggestion-job-2025-02-04-21-00-21-325                                       

............2025-02-04 21:02:24.712349: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2025-02-04 21:02:24.712382: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2025-02-04 21:02:26.407057: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2025-02-04 21:02:26.407087: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2025-02-04 21:02:26.407111: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ip-10-2-234-104.ec2.internal): /proc/driver/nvidia/version does not exist
2025-02-04 21:02:26.407390: I tensor

In [30]:
# Verify that the expected files were created in baseline job
# You can also do this by looking in your s3 bucket for the expected files

# List objects in the baseline output prefix
response = s3.list_objects_v2(Bucket=bucket)#, Prefix=prefix)

# Print the files in the output directory
print("Files in baseline output directory:")
for obj in response.get('Contents', []):
    print(obj['Key'])

# Download and print the content of statistics.json
statistics_key = "statistics.json"

statistics_obj = s3.get_object(Bucket=bucket, Key=statistics_key)
statistics_content = statistics_obj['Body'].read().decode('utf-8')
statistics_data = json.loads(statistics_content)
print("Statistics Data:", json.dumps(statistics_data, indent=4))

# Download and print the content of constraints.json
constraints_key = "constraints.json"
constraints_obj = s3.get_object(Bucket=bucket, Key=constraints_key)
constraints_content = constraints_obj['Body'].read().decode('utf-8')
constraints_data = json.loads(constraints_content)
print("Constraints Data:", json.dumps(constraints_data, indent=4))

Files in baseline output directory:
constraints.json
iris.csv
models/dmg-iris-model.joblib
statistics.json


[02/04/25 21:14:08] INFO     Skipping checksum validation. Response did not contain one of the  ]8;id=618850;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/httpchecksum.py\httpchecksum.py]8;;\:]8;id=246237;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/httpchecksum.py#481\481]8;;\
                             following algorithms: ['crc32', 'sha1', 'sha256'].                                    

Statistics Data: {
    "version": 0.0,
    "dataset": {
        "item_count": 150
    },
    "features": [
        {
            "name": "sepal_length",
            "inferred_type": "Fractional",
            "numerical_statistics": {
                "common": {
                    "num_present": 150,
                    "num_missing": 0
                },
                "mean": 5.843333333333335,
                "sum": 876.5000000000002,
                "std_dev": 0.8253012917851417,
                "min": 4.3,
                "max": 7.9,
                "approximate_num_distinct_values": 33,
                "completeness": 1.0,
                "distribution": {
                    "kll": {
                        "buckets": [
                            {
                                "lower_bound": 4.3,
                                "upper_bound": 4.66,
                                "count": 9.0
                            },
                            {
                     

                    INFO     Skipping checksum validation. Response did not contain one of the  ]8;id=183474;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/httpchecksum.py\httpchecksum.py]8;;\:]8;id=790894;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/httpchecksum.py#481\481]8;;\
                             following algorithms: ['crc32', 'sha1', 'sha256'].                                    

Constraints Data: {
    "version": 0.0,
    "features": [
        {
            "name": "sepal_length",
            "inferred_type": "Fractional",
            "completeness": 1.0,
            "num_constraints": {
                "is_non_negative": true
            }
        },
        {
            "name": "sepal_width",
            "inferred_type": "Fractional",
            "completeness": 1.0,
            "num_constraints": {
                "is_non_negative": true
            }
        },
        {
            "name": "petal_length",
            "inferred_type": "Fractional",
            "completeness": 1.0,
            "num_constraints": {
                "is_non_negative": true
            }
        },
        {
            "name": "petal_width",
            "inferred_type": "Fractional",
            "completeness": 1.0,
            "num_constraints": {
                "is_non_negative": true
            }
        },
        {
            "name": "species",
            "inferred_t

In [38]:
# Set up Monitoring Schedule
# Set up a monitoring schedule to periodically analyze the data being sent to your model and detect any deviations from the baseline.
# Once the baseline job is complete and the baseline constraints are available, you can set up a monitoring schedule.
# periodically analyze the data being sent to your model endpoint and detect any deviations from the baseline.
# 2 steps:
# Define the Monitoring Schedule: This step involves setting up the monitoring schedule to periodically analyze the data being sent to your model endpoint.
# Configure Data Capture: Ensure that data capture is enabled for your endpoint to collect incoming requests and responses.

# Initialize SageMaker session and role
sagemaker_session = Session()
role = get_execution_role()

# Define variables
endpoint_name = 'dmg-iris-endpoint'
endpoint_config_name = 'dmg-iris-endpoint-config'
monitoring_schedule_name = 'dmg-iris-monitoring-schedule'
monitoring_output_uri = 's3://dmg-iris-hakkoda/monitoring-output'
baseline_dataset_uri = 's3://dmg-iris-hakkoda/iris.csv'
baseline_output_uri = 's3://dmg-iris-hakkoda'

# Create a default model monitor instance
model_monitor = DefaultModelMonitor(
    role=role,
    sagemaker_session=sagemaker_session
)

# Set up data capture configuration
data_capture_config = DataCaptureConfig(
    destination_s3_uri=monitoring_output_uri,
    enable_capture=True,
    sampling_percentage=100,
    capture_options=["REQUEST", "RESPONSE"]
)

# Create a new endpoint configuration with data capture enabled
sagemaker_client = boto3.client('sagemaker')
response = sagemaker_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            'VariantName': 'AllTraffic',
            'ModelName': 'dmg-iris-model',
            'InstanceType': 'ml.m5.large',
            'InitialInstanceCount': 1,
        },
    ],
    DataCaptureConfig={
        'EnableCapture': True,
        'InitialSamplingPercentage': 100,
        'DestinationS3Uri': monitoring_output_uri,
        'CaptureOptions': [
            {'CaptureMode': 'Input'},
            {'CaptureMode': 'Output'},
        ],
    }
)

# Update the endpoint to use the new endpoint configuration
sagemaker_client.update_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=endpoint_config_name
)

# Create monitoring schedule
model_monitor.create_monitoring_schedule(
    endpoint_input=endpoint_name,
    output=monitoring_output_uri,
    # schedule_cron_expression="cron(0 0 * * ? *)",  # This cron expression represents daily monitoring
    schedule_cron_expression="cron(0 0 1 */3 ? *)",  # This cron expression represents every 3 months
    monitoring_schedule_name=monitoring_schedule_name,
    ground_truth_input=baseline_dataset_uri
)

[02/04/25 21:25:28] INFO     Ignoring unnecessary instance type: None.                            ]8;id=672950;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=715221;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/image_uris.py#528\528]8;;\

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:37                                                                                   │
│                                                                                                  │
│   34                                                                                             │
│   35 # Create a new endpoint configuration with data capture enabled                             │
│   36 sagemaker_client = boto3.client('sagemaker')                                                │
│ ❱ 37 response = sagemaker_client.create_endpoint_config(                                         │
│   38 │   EndpointConfigName=endpoint_config_name,                                                │
│   39 │   ProductionVariants=[                                                                    │
│   40 │   │   {                                                                                   │
│                                                                                                  │
│ /home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/client.py:569 in     │
│ _api_call                                                                                        │
│                                                                                                  │
│    566 │   │   │   │   │   f"{py_operation_name}() only accepts keyword arguments."              │
│    567 │   │   │   │   )                                                                         │
│    568 │   │   │   # The "self" in this scope is referring to the BaseClient.                    │
│ ❱  569 │   │   │   return self._make_api_call(operation_name, kwargs)                            │
│    570 │   │                                                                                     │
│    571 │   │   _api_call.__name__ = str(py_operation_name)                                       │
│    572                                                                                           │
│                                                                                                  │
│ /home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/client.py:1023 in    │
│ _make_api_call                                                                                   │
│                                                                                                  │
│   1020 │   │   │   │   "Code"                                                                    │
│   1021 │   │   │   )                                                                             │
│   1022 │   │   │   error_class = self.exceptions.from_code(error_code)                           │
│ ❱ 1023 │   │   │   raise error_class(parsed_response, operation_name)                            │
│   1024 │   │   else:                                                                             │
│   1025 │   │   │   return parsed_response                                                        │
│   1026                                                                                           │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ClientError: An error occurred (ValidationException) when calling the CreateEndpointConfig operation: Could not 
find model "dmg-iris-model".